In [1]:
import os
import requests
import pandas as pd
from datetime import datetime

# ---------- CONFIG ----------
API_URL = "https://api.chnwt.dev/thai-gold-api/latest"
RAW_FILE = "data/raw/ingest_gold.csv"
# ----------------------------

def fetch_and_save_gold_data():
    os.makedirs(os.path.dirname(RAW_FILE), exist_ok=True)

    resp = requests.get(API_URL)
    resp.raise_for_status()
    data = resp.json()

    response = data.get("response", {})
    price = response.get("price", {})

    gold = price.get("gold", {})
    gold_bar = price.get("gold_bar", {})
    change = price.get("change", {})

    # รวมข้อมูลทั้งหมดใน 1 แถว
    row = {
        "date": response.get("date"),
        "update_time": response.get("update_time"),
        "gold_buy": gold.get("buy"),
        "gold_sell": gold.get("sell"),
        "gold_bar_buy": gold_bar.get("buy"),
        "gold_bar_sell": gold_bar.get("sell"),
        "change_prev": change.get("compare_previous"),
        "change_yesterday": change.get("compare_yesterday"),
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    }

    df_new = pd.DataFrame([row])

    # ถ้ามีไฟล์อยู่แล้ว → อ่านของเก่ามาต่อท้าย
    if os.path.exists(RAW_FILE):
        df_old = pd.read_csv(RAW_FILE)
        df_all = pd.concat([df_old, df_new], ignore_index=True)
    else:
        df_all = df_new

    # เขียนทับไฟล์เดิม (แต่ข้อมูลถูก append แล้ว)
    df_all.to_csv(RAW_FILE, index=False, encoding="utf-8-sig")

    print(f"✅ Updated data saved to {RAW_FILE}")
    print(df_new)

if __name__ == "__main__":
    fetch_and_save_gold_data()


✅ Updated data saved to data/raw/ingest_gold.csv
             date    update_time   gold_buy  gold_sell gold_bar_buy  \
0  18 ตุลาคม 2568  เวลา 09:14 น.  66,800.00  64,581.60    66,000.00   

  gold_bar_sell change_prev change_yesterday            timestamp  
0     65,900.00      -1,200           -1,200  2025-10-18 17:11:11  


In [1]:
import pandas as pd

RAW_FILE = "data/raw/ingest_gold.csv"

df = pd.read_csv(RAW_FILE)
df.drop_duplicates(subset=["date", "update_time"], keep="last", inplace=True)
df.to_csv(RAW_FILE, index=False, encoding="utf-8-sig")

print("✅ Removed duplicates permanently.")


✅ Removed duplicates permanently.


In [2]:
import pandas as pd

# ระบุ path ของไฟล์ CSV ที่ต้องการแก้
csv_path = r"C:\Users\User\Documents\TestDS\data\raw\ingest_gold.csv"

# อ่านข้อมูลทั้งหมด
df = pd.read_csv(csv_path)

# แสดงจำนวนแถวก่อนลบ
print(f"ก่อนลบ: {len(df)} แถว")

# ลบแถวสุดท้าย
df = df.iloc[:-1, :]

# เขียนกลับลงไฟล์เดิม (ทับไฟล์เก่า)
df.to_csv(csv_path, index=False, encoding="utf-8-sig")

print(f"หลังลบ: {len(df)} แถว")
print("✅ ลบข้อมูลล่าสุดถาวรเรียบร้อยแล้ว")


ก่อนลบ: 5 แถว
หลังลบ: 4 แถว
✅ ลบข้อมูลล่าสุดถาวรเรียบร้อยแล้ว
